In [1]:
import os,sys
import numpy as np
import pandas as pd
# from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
# from tqdm import tqdm
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostRegressor
import gc
from  numba import cuda


print_to_file = False 
test_run = False 

# train = pd.read_pickle(ENV.lightgbm_train_764.value)
train = pd.read_pickle(ENV.lgb_train_0827_na_extrem.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(ENV.lightgbm_test_764.value)
test = pd.read_pickle(ENV.lgb_test_0827_na_extrem.value)
print('test shape is: {}'.format(test.shape))
fe_id = 'comb_764'

train shape is: (307511, 409)
test shape is: (48744, 408)


In [2]:
train['SK_ID_CURR'] = train['SK_ID_CURR'].astype(int)
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype(int)
targets = train.TARGET.values

train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

X = pd.concat([train.drop('TARGET',axis=1),test])

In [3]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [4]:
def fill_na_list(X,th=200):
    re = scan_nan_portion(X)
#     re = re[re<0.15]
    re = re.sort_values()
    col_list = []
    na_list = []
    for col in re.index:
        if re[col] > 0 and X[col].nunique() >= th and str(X[col].dtypes)!= 'category' and str(X[col].dtypes)!= 'object':
            col_list.append(col)
            na_list.append(re[col])
    return col_list,na_list
            
def check_category_col(X,th=200):
    cate = []
    cols = list(X.columns)
    for col in cols:
        if X[col].nunique() < th or str(X[col].dtypes)== 'category' or str(X[col].dtypes)== 'object':
            cate.append(cols)
    return cate


def cate_index(df,category):
    index_list = []
    for col in df.columns:
        if col in category:
            index_list.append(category.index(col))
    return index_list
    
def trainCAT(na_train_x,na_val_x,na_train_y,na_val_y):
    cat_col = cate_index(na_train_x,categorys)
    model = CatBoostRegressor(iterations=4500, 
                              depth=None, 
                              thread_count=10,
                              learning_rate=0.01, 
                              loss_function='RMSE',
                              verbose=500,
                              task_type='GPU')
    model.fit(na_train_x,na_train_y,
              cat_features=cat_col,
              eval_set=[(na_val_x, na_val_y)],
              early_stopping_rounds=3,
              verbose_eval=500,
              metric_period=500)

#     reg.fit(na_train_x, na_train_y, eval_set=[(na_train_x,na_train_y),(na_val_x, na_val_y)],  verbose=200,early_stopping_rounds=100,eval_metric='l1' )
    return model

def saving(X):
    print('saving...')
    train_save = X.iloc[:307511].copy()
    train_save['TARGET'] = targets
    print(train_save.shape)

    test_save = X.iloc[307511:].copy()
    train_save.to_pickle(ENV.lgb_train_0827_na_extrem.value)
    test_save.to_pickle(ENV.lgb_test_0827_na_extrem.value)
    
    


def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2))

In [5]:
col_list,na_list = fill_na_list(X)
categorys = check_category_col(X)

In [6]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col))
val = 0.1
save_count = 1
count = 0



for col,nav in zip(col_list,na_list):
    try:
        saved_df = pd.read_csv('ratio.csv')
    except:
        saved_df = pd.DataFrame()
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
#     feature_col = list(set(feature_col_base + Processed_list))
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    break
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainCAT(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    print('####################')
    meanNA = np.mean(na_train_y) * np.ones(len(na_val_y))
    preds = reg.predict(na_val_x)
    me_rmse = rmse(meanNA,na_val_y)
    es_rmse = rmse(preds,na_val_y)
    corr = np.corrcoef(preds,na_val_y).flatten()[1]
    print('mean encoding: {}'.format(me_rmse))
    print('estimate encoding: {}'.format(es_rmse))
    print('me/estimate ratio is : {}'.format(me_rmse/es_rmse))
    print('correlation in val: {}'.format(corr))
    new_df = pd.DataFrame({'meanEncoRMSE':[me_rmse],
                           'catBoostEncoRMSE':[es_rmse],
                           'meByCatRatio':[me_rmse/es_rmse],
                           'colName':[col],
                           'corr':[corr],
                           'preds_mean':[np.mean(preds)],
                           'preds_std':[np.std(preds)],
                           'val_mean':[np.mean(na_val_y)],
                           'val_std':[np.std(na_val_y)],
                           'naRatio':[nav]})
    conc = pd.concat([saved_df,new_df])
    conc.to_csv('ratio.csv',index=False)
    print('####################')
    del reg
    gc.collect()
    

start processing bureau_AMT_CREDIT_SUM_DEBT_divide_AMT_CREDIT_SUM_LIMIT_max 
the na is:0.21264543655527643


In [ ]:
#1. all null
#2. train null, test no null

In [9]:
na_train_x.isnull().sum()

prev_AMT_APPLICATION_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_max                   0
last_10by100_fraction_installment_paid_late_in_days_sum                                  0
CLOSED_DAYS_CREDIT_UPDATE_MEAN                                                       28485
NAME_EDUCATION_TYPE_CODE_GENDER_CNT_CHILDREN_max_diff                                    4
REGION_POPULATION_RELATIVE_x                                                             0
bureau_AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_ENDDATE_mean                               0
bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_mean                         65698
PREV_CHANNEL_TYPE_Channel of corporate sales_MEAN                                    11759
inst_RATE_AMT_CUM_PAYMENT_min                                                            0
prev_AMT_APPLICATION_divide_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_TERMINATION_max         0
bureau_AMT_CREDIT_SUM_divide_DAYS_CREDIT_min                                             0

In [16]:
np.corrcoef(preds,na_val_y).flatten()

In [20]:
s.flatten()[1]

0.003132356121393304

In [7]:
np.mean(preds)

7.332347094563894